In [1]:
import os 
from dotenv import load_dotenv
import json

from pathlib import Path

from mistralai import Mistral


import json
import os
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Set

from dotenv import load_dotenv

# Add the project root to Python path for imports
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from bill_parser_engine.core.reference_resolver.pipeline import BillProcessingPipeline

load_dotenv('../.env.local')

api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key)

In [2]:
# Define file paths
bill_file = project_root / "data" / "legal_bill" / "sampled_legislative_bill.md"
output_dir = project_root / "scripts " / "output"
log_file_path = project_root / "notebooks" / f"detailed_reconstruction_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"


In [3]:
pipeline = BillProcessingPipeline(use_cache=True, log_file_path=log_file_path)

2025-07-24 16:50:30,852 - bill_parser_engine.core.reference_resolver.instruction_decomposer - INFO - InstructionDecomposer initialized with caching: enabled
2025-07-24 16:50:30,867 - bill_parser_engine.core.reference_resolver.operation_applier - INFO - OperationApplier initialized with caching: enabled
2025-07-24 16:50:30,885 - bill_parser_engine.core.reference_resolver.result_validator - INFO - ResultValidator initialized with caching: enabled
2025-07-24 16:50:30,913 - bill_parser_engine.core.reference_resolver.pipeline - INFO - BillProcessingPipeline initialized with centralized Mistral API caching: enabled
2025-07-24 16:50:30,914 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Detailed reconstruction logging configured: /Users/duphan/Projects/bill-parser-engine/notebooks/detailed_reconstruction_log_20250724_165030.txt (will be created when first needed)


In [4]:
pipeline.load_legislative_text_from_file(bill_file)

2025-07-24 16:50:30,920 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Loaded legislative text: 9991 characters
2025-07-24 16:50:30,920 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Loaded legislative text from file: /Users/duphan/Projects/bill-parser-engine/data/legal_bill/sampled_legislative_bill.md


In [5]:
#pipeline.clear_component_cache('target_identifier')
#pipeline.clear_component_cache("original_text_retriever")
#pipeline.clear_cache("legal_amendment_reconstructor_focused")
# Clear cache for all components that are part of the reconstruction process
#pipeline.clear_cache("instruction_decomposer")
#pipeline.clear_cache("operation_applier") 
#pipeline.clear_cache("result_validator")

pipeline.clear_cache("reference_locator")
pipeline.clear_cache('reference_object_linker')

2025-07-24 16:50:30,930 - bill_parser_engine.core.reference_resolver.cache_manager - INFO - Cleared 18 Mistral API cache entries for component 'reference_locator'
2025-07-24 16:50:30,930 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Cleared 18 Mistral API cache entries for component 'reference_locator'
2025-07-24 16:50:30,931 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Cleared 0 Mistral API cache entries for component 'reference_object_linker'


0

In [6]:
chunks = pipeline.step_1_split_bill()

2025-07-24 16:50:30,937 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 1: Splitting legislative bill into chunks...
2025-07-24 16:50:30,943 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Split into 22 chunks


In [7]:
chunks[15]

BillChunk(text="a) Au premier alinéa, les mots : « , et notamment la désignation de l'autorité administrative, les conditions de délivrance, de renouvellement, de suspension, de modulation et de retrait des agréments, des certificats ainsi que des habilitations des organismes » sont supprimés ;", titre_text='# TITRE Iᴱᴿ', article_label='Article 1ᵉʳ', article_introductory_phrase='Le code rural et de la pêche maritime est ainsi modifié :', major_subdivision_label=None, major_subdivision_introductory_phrase=None, numbered_point_label='6°', numbered_point_introductory_phrase="L'article L. 254-7-1 est ainsi modifié :", lettered_subdivision_label='a)', hierarchy_path=['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '6°', 'a)'], chunk_id='# TITRE Iᴱᴿ::Article 1ᵉʳ::6°::a)', start_pos=6886, end_pos=7171, target_article=None, inherited_target_article=TargetArticle(operation_type=<TargetOperationType.MODIFY: 'MODIFY'>, code='code rural et de la pêche maritime', article='L. 254-7-1'))

In [8]:
target_results = pipeline.step_2_identify_target_articles()

2025-07-24 16:50:30,953 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 2: Identifying target articles for 22 chunks...
2025-07-24 16:50:30,963 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Target identification complete: 14 unique articles identified


✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier
✓ Using cached result for TargetArticleIdentifier


In [9]:
target_results

[{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::2°::a)',
  'chunk_text_preview': 'a) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés par les mo...',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '2°', 'a)'],
  'target_article': {'operation_type': 'MODIFY',
   'code': 'code rural et de la pêche maritime',
   'article': 'L. 254-1',
   'full_citation': 'code rural et de la pêche maritime::L. 254-1',
   'confidence': 1.0,
   'raw_text': 'a) Au 3° du II, les mots : « prévu aux articles L....'}},
 {'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::2°::b)',
  'chunk_text_preview': "b) Le VI est ainsi rédigé : « VI. – L'exercice de l'activité de conseil mentionnée au 3° du II est i...",
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '2°', 'b)'],
  'target_article': {'operation_type': 'MODIFY',
   'code': 'code rural et de la pêche maritime',
   'article': 'L. 254-1',
   'full_citation': 'code rural et de la pêche maritime::L. 254-1',
   'confidence': 1.0,
   'raw_

In [10]:
weird_issues = []
for res in target_results:
    if res.get('target_article').get('code') is None: 
        print(res)
        weird_issues.append(res)
        print('----')

In [11]:
len(weird_issues)

0

In [12]:
len(target_results)

22

In [13]:
retrieval_results = pipeline.step_3_retrieve_original_texts()
retrieval_results

2025-07-24 16:50:30,991 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 3: Retrieving original text for target articles...
2025-07-24 16:50:30,993 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - ✓ Retrieved article L. 254-1 from cache
2025-07-24 16:50:30,994 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - ✓ Retrieved article L. 254-1-1 from cache
2025-07-24 16:50:30,995 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - ✓ Retrieved article L. 254-1-2 from cache
2025-07-24 16:50:30,996 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - ✓ Retrieved article L. 254-1-3 from cache
2025-07-24 16:50:30,996 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - ✓ Retrieved article L. 254-2 from cache
2025-07-24 16:50:30,997 - bill_parser_engine.core.reference_resolver.original_text_retriever - INFO - ✓ Retrieved article L. 254-6-2 from cac

[{'article_key': 'code rural et de la pêche maritime::L. 254-1',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1',
  'operation_type': 'MODIFY',
  'original_text': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant pour leur compte, y compris les groupements d'achats ; 2° L'application, en qualité de prestataire de services, des produits phytopharmaceutiques, sauf si elle est effectuée dans le cadre de contrats d'entraide à titre gratuit au sens de l'article L. 325-1 ou par un exploitant agricole titulaire du certificat mentionné au II de l'article L. 254-3 sur des exploitations dont la surface agricole utile est infé

In [14]:
pipeline._analyze_retrieval_results(retrieval_results)

{'total_articles': 14,
 'successful_retrievals': 14,
 'failed_retrievals': 0,
 'success_rate': 1.0,
 'source_stats': {'cache': 13, 'insert_operation': 1},
 'text_length_stats': {'average': 1403.9285714285713,
  'maximum': 3321,
  'minimum': 0,
  'total_characters': 19655},
 'failed_articles': []}

In [15]:
reconstruction_results = pipeline.step_4_reconstruct_texts()


2025-07-24 16:50:31,013 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 4: Applying text reconstruction...
2025-07-24 16:50:31,014 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Processing 22 chunks with identified target articles (skipping 0 chunks with pure versioning metadata or errors)
2025-07-24 16:50:31,015 - bill_parser_engine.core.reference_resolver.legal_amendment_reconstructor - INFO - Cache temporarily disabled for chunk # TITRE Iᴱᴿ::Article 1ᵉʳ::2°::a) to force fresh processing
2025-07-24 16:50:31,015 - bill_parser_engine.core.reference_resolver.legal_amendment_reconstructor - INFO - Step 1: Decomposing amendment instruction for chunk # TITRE Iᴱᴿ::Article 1ᵉʳ::2°::a)
2025-07-24 16:50:31,016 - bill_parser_engine.core.reference_resolver.instruction_decomposer - INFO - Parsing amendment instruction: a) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés par les mots : « à l'utilisation des produits phytopha

In [16]:
pipeline._analyze_reconstruction_results(reconstruction_results)#.get('failed_chunks')

{'total_chunks': 22,
 'successful_reconstructions': 18,
 'failed_reconstructions': 4,
 'success_rate': 0.8181818181818182,
 'operation_type_stats': {'MODIFY': 14, 'INSERT': 3, 'ABROGATE': 1},
 'text_length_stats': {'average_deleted_length': 50.72222222222222,
  'average_newly_inserted_length': 225.22222222222223,
  'average_after_state_length': 1624.888888888889,
  'max_deleted_length': 225,
  'max_newly_inserted_length': 1344,
  'max_after_state_length': 3562,
  'total_deleted_characters': 913,
  'total_newly_inserted_characters': 4054,
  'total_after_state_characters': 29248},
 'failure_analysis': {'no_reconstruction_result': 0,
  'reconstruction_failed': 4},
 'operation_failure_stats': {},
 'failed_chunks': ['# TITRE Iᴱᴿ::Article 1ᵉʳ::3° bis::a)',
  '# TITRE Iᴱᴿ::Article 1ᵉʳ::3° quater::a)',
  '# TITRE Iᴱᴿ::Article 1ᵉʳ::5° ter::a)',
  '# TITRE Iᴱᴿ::Article 1ᵉʳ::6° bis::b)']}

In [17]:
reconstruction_results[0]

{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::2°::a)',
 'chunk_text_preview': 'a) Au 3° du II, les mots : « prévu aux articles L. 254-6-2 et L. 254-6-3 » sont remplacés par les mo...',
 'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '2°', 'a)'],
 'target_article': {'operation_type': 'MODIFY',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1',
  'confidence': 1.0,
  'raw_text': 'a) Au 3° du II, les mots : « prévu aux articles L....'},
 'reconstruction_result': {'deleted_or_replaced_text': 'prévu aux articles L. 254-6-2 et L. 254-6-3',
  'newly_inserted_text': "à l'utilisation des produits phytopharmaceutiques",
  'intermediate_after_state_text': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux ut

In [18]:
failed_reconstruction = []
for res in reconstruction_results:
    if res.get('chunk_id') in pipeline._analyze_reconstruction_results(reconstruction_results).get('failed_chunks'):
        failed_reconstruction.append(res)

In [19]:
failed_reconstruction

[{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::3° bis::a)',
  'chunk_text_preview': 'a) Le I est ainsi modifié : à la fin du 1°, les mots : « mentionnée aux 1° ou 2° du même II ou au IV...',
  'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '3° bis', 'a)'],
  'target_article': {'operation_type': 'MODIFY',
   'code': 'code rural et de la pêche maritime',
   'article': 'L. 254-1',
   'confidence': 1.0,
   'raw_text': 'a) Le I est ainsi modifié : à la fin du 1°, les mo...'},
  'reconstruction_result': {'deleted_or_replaced_text': '',
   'newly_inserted_text': '',
   'intermediate_after_state_text': "I.-Les produits phytopharmaceutiques mentionnés au présent chapitre sont ceux définis au 1 de l'article 2 du règlement (CE) n° 1107/2009. II.-Est subordonné à la détention d'un agrément l'exercice des activités suivantes : 1° La mise en vente, la vente ou la distribution à titre gratuit des produits phytopharmaceutiques aux utilisateurs de ces produits ou aux personnes physiques ou morales agissant

In [20]:
len(failed_reconstruction)

4

In [21]:
located_ref = pipeline.step_5_locate_references()

2025-07-24 16:50:31,223 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 5: Locating normative references using focused scanning...
2025-07-24 16:50:31,224 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Processing 18 successful reconstructions (skipping 4 failed reconstructions)
2025-07-24 16:50:31,225 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Processing delta fragments: 92 chars total
2025-07-24 16:50:32,319 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:50:32,326 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 1 references: 1 DELETIONAL, 0 DEFINITIONAL
2025-07-24 16:50:32,328 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Cached result for future use
2025-07-24 16:50:32,329 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reconstruction 1/18: Located 1 references (1 DELETIONAL

⏱️ ReferenceLocator: Waiting 2.4s for rate limiting...


2025-07-24 16:50:38,768 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:50:38,774 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 5 references: 0 DELETIONAL, 5 DEFINITIONAL
2025-07-24 16:50:38,777 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Cached result for future use
2025-07-24 16:50:38,777 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reconstruction 2/18: Located 5 references (0 DELETIONAL, 5 DEFINITIONAL) in 6.45s with 3.9x efficiency gain
2025-07-24 16:50:38,778 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Processing delta fragments: 254 chars total
2025-07-24 16:50:41,634 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:50:41,639 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 6 references: 4 DELETIONAL, 2 DEFINI

⏱️ ReferenceLocator: Waiting 0.6s for rate limiting...


2025-07-24 16:50:44,808 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:50:44,814 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 7 references: 5 DELETIONAL, 2 DEFINITIONAL
2025-07-24 16:50:44,817 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Cached result for future use
2025-07-24 16:50:44,818 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reconstruction 4/18: Located 7 references (5 DELETIONAL, 2 DEFINITIONAL) in 3.17s with 4.5x efficiency gain
2025-07-24 16:50:44,819 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Processing delta fragments: 16 chars total


...

2025-07-24 16:51:24,623 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:51:24,629 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 0 references: 0 DELETIONAL, 0 DEFINITIONAL
2025-07-24 16:51:24,632 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Cached result for future use
2025-07-24 16:51:24,632 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reconstruction 16/18: Located 0 references (0 DELETIONAL, 0 DEFINITIONAL) in 2.24s with 67.1x efficiency gain
2025-07-24 16:51:24,633 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Processing delta fragments: 1344 chars total


⏱️ ReferenceLocator: Waiting 3.2s for rate limiting...


2025-07-24 16:51:28,653 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:51:28,659 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 1 references: 0 DELETIONAL, 1 DEFINITIONAL
2025-07-24 16:51:28,661 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Cached result for future use
2025-07-24 16:51:28,662 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reconstruction 17/18: Located 1 references (0 DELETIONAL, 1 DEFINITIONAL) in 4.03s with 1.0x efficiency gain
2025-07-24 16:51:28,662 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Processing delta fragments: 79 chars total


⏱️ ReferenceLocator: Waiting 2.7s for rate limiting...


2025-07-24 16:51:32,219 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:51:32,224 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Located 1 references: 1 DELETIONAL, 0 DEFINITIONAL
2025-07-24 16:51:32,226 - bill_parser_engine.core.reference_resolver.reference_locator - INFO - Cached result for future use
2025-07-24 16:51:32,227 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reconstruction 18/18: Located 1 references (1 DELETIONAL, 0 DEFINITIONAL) in 3.56s with 3.7x efficiency gain
2025-07-24 16:51:32,228 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 5 completed: Located 34 references across 18 successful chunks (skipped 4 failed reconstructions) using focused scanning approach


In [23]:
pipeline._analyze_reference_location_results(located_ref)

{'total_chunks': 22,
 'successful_locations': 18,
 'failed_locations': 4,
 'success_rate': 0.8181818181818182,
 'reference_stats': {'total_references': 34,
  'deletional_references': 16,
  'definitional_references': 18,
  'average_references_per_chunk': 1.8888888888888888,
  'average_confidence': 0.9167647058823529,
  'confidence_distribution': {'high_confidence': 33,
   'medium_confidence': 1,
   'low_confidence': 0}},
 'focused_scanning_performance': {'average_performance_gain': 1.0,
  'max_performance_gain': 1.0,
  'min_performance_gain': 1.0,
  'total_performance_gains': []},
 'failed_chunks': ['# TITRE Iᴱᴿ::Article 1ᵉʳ::3° bis::a)',
  '# TITRE Iᴱᴿ::Article 1ᵉʳ::3° quater::a)',
  '# TITRE Iᴱᴿ::Article 1ᵉʳ::5° ter::a)',
  '# TITRE Iᴱᴿ::Article 1ᵉʳ::6° bis::b)']}

In [24]:
located_ref[3]

{'chunk_id': '# TITRE Iᴱᴿ::Article 1ᵉʳ::3° ter::a)',
 'chunk_text_preview': 'a) Le premier alinéa est ainsi modifié : les mots : « mentionnée aux 1° ou 2° du même II ou au IV du...',
 'hierarchy_path': ['# TITRE Iᴱᴿ', 'Article 1ᵉʳ', '3° ter', 'a)'],
 'target_article': {'operation_type': 'MODIFY',
  'code': 'code rural et de la pêche maritime',
  'article': 'L. 254-1-2',
  'confidence': 1.0,
  'raw_text': 'a) Le premier alinéa est ainsi modifié : les mots ...'},
 'reconstruction_result': {'deleted_or_replaced_text': 'mentionnée aux 1° ou 2° du même II ou au IV du même article mentionnée aux 1° ou 2° de ce II ou à ce IV de ce même article de ce II',
  'newly_inserted_text': 'de producteur au sens du premier alinéa du VI du même article L. 254-1 de producteur au sens du même premier alinéa du II du même article L. 254-1',
  'intermediate_after_state_text': "Un membre d'un organe de surveillance, d'administration et de direction d'une personne morale exerçant une activité mentionnée au 3° 

In [25]:
ref_link = pipeline.step_6_link_references()

2025-07-24 16:52:16,550 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Step 6: Linking references to grammatical objects...
2025-07-24 16:52:16,552 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Processing 14 successful reference locations (skipping 8 failed/skipped locations)
2025-07-24 16:52:18,813 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:52:18,818 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.900) - skipping iteration for reference: prévu aux articles L. 254-6-2 et L. 254-6-3
2025-07-24 16:52:18,820 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked reference: prévu aux articles L. 254-6-2 et L. 254-6-3 → conseil
2025-07-24 16:52:18,820 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked 1 out of 1 references
2025-07-24 16:52:18,821 - bill_p

⏱️ ReferenceObjectLinker: Waiting 1.2s for rate limiting...


2025-07-24 16:52:22,295 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:52:22,301 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.900) - skipping iteration for reference: au 3° du II
2025-07-24 16:52:22,303 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked reference: au 3° du II → activité de conseil


⏱️ ReferenceObjectLinker: Waiting 1.3s for rate limiting...


2025-07-24 16:52:26,083 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:52:26,087 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.950) - skipping iteration for reference: du 11 de l'article 3 du règlement (CE) n° 1107/2009
2025-07-24 16:52:26,089 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked reference: du 11 de l'article 3 du règlement (CE) n° 1107/2009 → producteur


...

2025-07-24 17:00:14,165 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 17:00:14,169 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.950) - skipping iteration for reference: mentionné au 3° du II de l'article L. 254-1
2025-07-24 17:00:14,171 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked reference: mentionné au 3° du II de l'article L. 254-1 → conseil
2025-07-24 17:00:14,172 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked 1 out of 1 references
2025-07-24 17:00:14,172 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reference location 11/14: Linked 1 references (0 DELETIONAL, 1 DEFINITIONAL) in 3.49s


⏱️ ReferenceObjectLinker: Waiting 1.3s for rate limiting...


2025-07-24 17:00:21,836 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 17:00:21,842 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.900) - skipping iteration for reference: mentionnées au IV de l'article L. 213-10-8 du code de l'environnement
2025-07-24 17:00:21,844 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked reference: mentionnées au IV de l'article L. 213-10-8 du code de l'environnement → personnes
2025-07-24 17:00:24,089 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 17:00:24,095 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.900) - skipping iteration for reference: mentionnées au 1° du II de l'article L. 254-1
2025-07-24 17:00:24,097 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - S

⏱️ ReferenceObjectLinker: Waiting 1.2s for rate limiting...


2025-07-24 17:00:30,170 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 17:00:30,175 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - High confidence (0.950) - skipping iteration for reference: au II de l'article L. 254-6-4
2025-07-24 17:00:30,178 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked reference: au II de l'article L. 254-6-4 → conseil stratégique à l'utilisation de produits phytopharmaceutiques
2025-07-24 17:00:30,178 - bill_parser_engine.core.reference_resolver.reference_object_linker - INFO - Successfully linked 1 out of 1 references
2025-07-24 17:00:30,179 - bill_parser_engine.core.reference_resolver.pipeline - INFO - Successful reference location 13/14: Linked 1 references (0 DELETIONAL, 1 DEFINITIONAL) in 6.08s
2025-07-24 17:00:32,792 - httpx - INFO - HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025

In [26]:
pipeline._analyze_reference_linking_results(ref_link)

{'total_chunks': 22,
 'successful_links': 14,
 'failed_links': 8,
 'success_rate': 0.6363636363636364,
 'reference_stats': {'total_links': 34,
  'deletional_links': 16,
  'definitional_links': 18,
  'average_links_per_chunk': 2.4285714285714284,
  'average_confidence': 0.9188235294117647,
  'confidence_distribution': {'high_confidence': 34,
   'medium_confidence': 0,
   'low_confidence': 0}},
 'object_analysis': {'total_objects_found': 34,
  'unique_objects_count': 21,
  'unique_objects': ["modalités d'application",
   'la part du capital',
   'personnes',
   'la liste',
   'certification',
   'lutte intégrée contre les ennemis des cultures',
   "la section IV de l'article L. 254-1",
   'section IV',
   'producteur',
   'substances de base'],
  'most_common_objects': [('producteur', 5),
   ('activité', 4),
   ('activités', 4),
   ('conseil', 2),
   ('la part du capital', 2),
   ("conseil stratégique à l'utilisation de produits phytopharmaceutiques", 2),
   ('activité de conseil', 1),
 

In [ ]:
ref_link[15]